# Data extraction: California Education Spreadsheets

Data sources are very diverse. Many official sources provide us with raw online data we can directly access with python libraries like [pandas](https://pandas.pydata.org/). We now show a series of examples on retrieving __Excel__ spreadsheets data from [SAT test results](https://www.cde.ca.gov/ds/sp/ai/). More packages [here](http://www.python-excel.org/) to work with Excel spreadsheets and python. For a comprehensive guide on how to load various types of data into pandas dataframes please check [IO tools](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

In [9]:
# perform web requests from an url
import requests

# xlrd helps us to extract data from Excel spreadsheets (.xls and .xlsx)
# https://xlrd.readthedocs.io/en/latest/
from xlrd import open_workbook

## California postsecondary preparation score results

Provides standardized test results from the SAT, ACT, and AP tests which measure high school students' achievement.

Source: [SAT, ACT, and AP Test Results](http://www.cde.ca.gov/ds/sp/ai/).


### The data file

- Direct link to the source data file: [http://www3.cde.ca.gov/researchfiles/satactap/sat19.xlsx](http://www3.cde.ca.gov/researchfiles/satactap/sat19.xlsx)

### Parsing and wrangling SAT data file

We will go through two different ways to work with Excel files. First, let's download the file: 

In [2]:
# download and save file locally
source_data_url_sat = "http://www3.cde.ca.gov/researchfiles/satactap/sat19.xlsx"
sat_file = requests.get(source_data_url_sat)

with open('sat19.xlsx', 'wb') as f:
    f.write(sat_file.content)

Now we are ready to open it with [`xlrd.open_workbook`](https://xlrd.readthedocs.io/en/latest/api.html):

In [3]:
book = open_workbook('sat19.xlsx')

xlrd allows us to open one of the xlsx sheets. It is also possible to get the heading row index by searching its columns labels:

In [4]:
# first sheet
sheet = book.sheets()[0]

# index of the heading row (one of the columns is named 'CDS')
first_row_idx = sheet.col_values(0).index('CDS')

# print the heading line 
print(sheet.row_values(first_row_idx))

['CDS', 'CCode', 'CDCode', 'SCode', 'RType', 'SName', 'DName', 'CName', 'Enroll12', 'NumTSTTakr12', 'NumERWBenchmark12', 'PctERWBenchmark12', 'NumMathBenchmark12', 'PctMathBenchmark12', 'Enroll11', 'NumTSTTakr11', 'NumERWBenchmark11', 'PctERWBenchmark11', 'NumMathBenchmark11', 'PctMathBenchmark11', 'TotNumBothBenchmark12', 'PctBothBenchmark12', 'TotNumBothBenchmark11', 'PctBothBenchmark11', 'Year']


It is also possible to print all the content of the file. Notice we discarded rows above the heading line. 

In [5]:
# python list of lists. Each xlsx row is packed into a list [...]
datarows =  [sheet.row_values(i) for i in range(first_row_idx+1, sheet.nrows)]
print(datarows)

[['06615980630046', '06', '0661598', '0630046', 'S', 'Colusa Alternative Home', 'Colusa Unified', 'Colusa', '18', 0.0, 'N/A', 'N/A', 'N/A', 'N/A', '18', 0.0, 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '2018-19'], ['06616060634758', '06', '0661606', '0634758', 'S', 'Maxwell Sr High', 'Maxwell Unified', 'Colusa', '29', 10.0, '*', '*', '*', '*', '26', 6.0, '*', '*', '*', '*', '*', '*', '*', '*', '2018-19'], ['19647331930924', '19', '1964733', '1930924', 'S', 'Belmont Senior High', 'Los Angeles Unified', 'Los Angeles', '206', 102.0, '31', '30.39', '14', '13.73', '219', 174.0, '42', '24.14', '12', '6.90', '14', '13.73', '11', '6.32', '2018-19'], ['19647331931476', '19', '1964733', '1931476', 'S', 'Canoga Park Senior High', 'Los Angeles Unified', 'Los Angeles', '227', 113.0, '54', '47.79', '18', '15.93', '333', 275.0, '97', '35.27', '37', '13.45', '18', '15.93', '35', '12.73', '2018-19'], ['19647331931856', '19', '1964733', '1931856', 'S', 'Whitman Continuation', 'Los Angeles Un

Now we are ready to load data it into a pandas dataframe.

In [6]:
# import pandas to use pandas DataFrame 
import pandas as pd 
pd.set_option("max_rows", 8)              

# create the dataframe
sat19 = pd.DataFrame(datarows, columns = sheet.row_values(first_row_idx)) 
sat19

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumTSTTakr11,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year
0,06615980630046,06,0661598,0630046,S,Colusa Alternative Home,Colusa Unified,Colusa,18,0.0,...,0.0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2018-19
1,06616060634758,06,0661606,0634758,S,Maxwell Sr High,Maxwell Unified,Colusa,29,10.0,...,6.0,*,*,*,*,*,*,*,*,2018-19
2,19647331930924,19,1964733,1930924,S,Belmont Senior High,Los Angeles Unified,Los Angeles,206,102.0,...,174.0,42,24.14,12,6.90,14,13.73,11,6.32,2018-19
3,19647331931476,19,1964733,1931476,S,Canoga Park Senior High,Los Angeles Unified,Los Angeles,227,113.0,...,275.0,97,35.27,37,13.45,18,15.93,35,12.73,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,57726780000000,57,5772678,0000000,D,N/A,Davis Joint Unified,Yolo,717,307.0,...,474.0,454,95.78,428,90.30,268,87.30,421,88.82,2018-19
2576,07000000000000,07,0700000,0000000,C,N/A,N/A,Contra Costa,14593,5412.0,...,5157.0,4583,88.87,3933,76.27,2721,50.28,3853,74.71,2018-19
2577,17000000000000,17,1700000,0000000,C,N/A,N/A,Lake,667,167.0,...,126.0,94,74.60,50,39.68,65,38.92,47,37.30,2018-19
2578,38000000000000,38,3800000,0000000,C,N/A,N/A,San Francisco,4447,2256.0,...,2042.0,1702,83.35,1566,76.69,1248,55.32,1482,72.58,2018-19


Another option is to directly load from the url into a pandas dataframe using [`pd.read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html):

In [7]:
# load the file, notice we still need to indicate the header row
# we can indicate column types using dtype={'CDS': object ...}
sat19_df = pd.read_excel(source_data_url_sat, header=first_row_idx, dtype={'CDS': object})  
sat19_df

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumTSTTakr11,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year
0,06615980630046,6,661598,630046,S,Colusa Alternative Home,Colusa Unified,Colusa,18,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
1,06616060634758,6,661606,634758,S,Maxwell Sr High,Maxwell Unified,Colusa,29,10,...,6,*,*,*,*,*,*,*,*,2018-19
2,19647331930924,19,1964733,1930924,S,Belmont Senior High,Los Angeles Unified,Los Angeles,206,102,...,174,42,24.14,12,6.90,14,13.73,11,6.32,2018-19
3,19647331931476,19,1964733,1931476,S,Canoga Park Senior High,Los Angeles Unified,Los Angeles,227,113,...,275,97,35.27,37,13.45,18,15.93,35,12.73,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,57726780000000,57,5772678,0,D,NaN,Davis Joint Unified,Yolo,717,307,...,474,454,95.78,428,90.30,268,87.30,421,88.82,2018-19
2576,07000000000000,7,700000,0,C,NaN,NaN,Contra Costa,14593,5412,...,5157,4583,88.87,3933,76.27,2721,50.28,3853,74.71,2018-19
2577,17000000000000,17,1700000,0,C,NaN,NaN,Lake,667,167,...,126,94,74.60,50,39.68,65,38.92,47,37.30,2018-19
2578,38000000000000,38,3800000,0,C,NaN,NaN,San Francisco,4447,2256,...,2042,1702,83.35,1566,76.69,1248,55.32,1482,72.58,2018-19
